In [1]:
%matplotlib inline

from __future__ import print_function

import sys
import numpy as np
import pandas as pd

from scipy.optimize import leastsq
import scipy.stats as stats
from scipy.stats import logistic as slogistic
from scipy.stats import norm as snorm
import io

from matplotlib import pyplot as plt
from matplotlib.cm import get_cmap
from mpl_toolkits.basemap import Basemap

import statsmodels.api as sm

from Utilities.loadData import loadTrackFile
from Utilities.config import ConfigParser
from Utilities.track import Track

import seaborn
seaborn.set_context("poster")
seaborn.set_style("whitegrid")

In [35]:
configstr = """
[DataProcess]
InputFile=C:/WorkSpace/data/TC/Allstorms.ibtracs_wmo.v03r06.csv
Source=IBTRACS
StartSeason=1901
FilterSeasons=True

[Region]
; Domain for windfield and hazard calculation
gridLimit={'xMin':90.,'xMax':160.,'yMin':-30.0,'yMax':-5.0}
gridSpace={'x':1.0,'y':1.0}
gridInc={'x':1.0,'y':0.5}

[Input]
landmask = C:/WorkSpace/tcrm/input/landmask.nc
mslpfile = C:/WorkSpace/data/MSLP/slp.day.ltm.nc
datasets = IBTRACS,LTMSLP

[IBTRACS]
; Input data file settings
url = ftp://eclipse.ncdc.noaa.gov/pub/ibtracs/v03r06/wmo/csv/Allstorms.ibtracs_wmo.v03r06.csv.gz
path = C:/WorkSpace/data/TC/
filename = C:/WorkSpace/data/TC/Allstorms.ibtracs_wmo.v03r06.csv
columns = tcserialno,season,num,skip,skip,skip,date,skip,lat,lon,skip,pressure
fielddelimiter = ,
numberofheadinglines = 3
pressureunits = hPa
lengthunits = km
dateformat = %Y-%m-%d %H:%M:%S
speedunits = kph

[LTMSLP]
; MSLP climatology file settings
URL = ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/surface/slp.day.1981-2010.ltm.nc
path = C:/WorkSpace/data/MSLP
filename = C:/WorkSpace/data/MSLP/slp.day.ltm.nc
"""

In [36]:
config = ConfigParser()
config.readfp(io.BytesIO(configstr))

In [37]:
trackFile = config.get('DataProcess', 'InputFile')
source = config.get('DataProcess', 'Source')

print("Track file: {0}".format(trackFile))
print("Track format: {0}".format(source))
tracks = loadTrackFile(configstr, trackFile, source)
print("There are {0:d} tracks in the input dataset".format(len(tracks)))

In [38]:
domain = config.geteval('Region', 'gridLimit')
mapkwargs = dict(llcrnrlon=domain['xMin'],
                 llcrnrlat=domain['yMin'],
                 urcrnrlon=domain['xMax'],
                 urcrnrlat=domain['yMax'],
                 resolution='c',
                 projection='cyl')
m = Basemap(**mapkwargs)

In [43]:
aprtracks = []
minprs = []
maxwind = []
datetimes = []
for n, track in enumerate(tracks):
    if track.inRegion(domain):
        if track.Month[0] == 4:
            idx = np.where(track.CentralPressure==track.trackMinPressure)[0]
            if track.Day[idx[0]] > 20:
                aprtracks.append(track)
                minprs.append(track.trackMinPressure)
                maxwind.append(track.trackMaxWind)
                datetimes.append(track.Datetime[idx[0]])
            
dts = [dt.strftime("%Y-%m-%d %H:%M:%S") for dt in datetimes]

In [45]:
print("Total {0} storms with minimum central pressure after April 20".format(len(dts)))

for dt, prs in zip(dts, minprs):
    print("{0}: {1} hPa".format(dt, prs))
    


In [47]:
from PlotInterface.tracks import saveTrackMap
from IPython.display import Image
startSeason = config.get("DataProcess", "StartSeason")
#fig = SingleTrackMap()
xx = np.arange(domain['xMin'], domain['xMax']+0.1,0.1)
yy = np.arange(domain['yMin'], domain['yMax']+0.1,0.1)

[xgrid, ygrid] = np.meshgrid(xx,yy)
title = "April TCs - {0} - 2013".format(startSeason)
mapkwargs = dict(llcrnrlon=domain['xMin']-20,
                     llcrnrlat=domain['yMin'],
                     urcrnrlon=domain['xMax'],
                     urcrnrlat=domain['yMax'],
                     resolution='f',
                     projection='merc')

saveTrackMap(aprtracks, xgrid, ygrid, title, mapkwargs, "tracks.png")
Image("tracks.png")